In [1]:
import sys
sys.path.insert(1, 'bin')
sys.path.insert(1, 'functional')

from persistence import sql
import functional as fn
import color as c
from sql import table

# Print all commands

In [ ]:
db = sql('functional')

for i in db.query('SELECT * FROM command'):
    print(c.blue + i['class'] + ':' + i['name'])
    print(c.white + fn.build(i['class'], i['name']))

db.close()

# Create command

In [ ]:
fn.create_command('python','if','''if({{condition}}):
  {{block}}''')

# Create command and add child commands

(this command does not have valid sintax)

In [15]:
fn.create_command('debug', 'test')
fn.set_child(('debug', 'test'),('python','if'),{'block':('select','from')})

In [6]:
fn.create_command('sql', 'delete where')
fn.set_child(('sql', 'delete where'),('sql','delete'))
fn.set_child(('sql', 'delete where'),('sql','where'))


# Example

In [3]:
print(fn.build('python','if',{'condition':'true','block':'print("hello world")\nprint("line2")'}))

if(true):
  print("hello world")
  print("line2")


# Example2

In [9]:
pessoa = table(
    'pessoa',
{
    'nome': 'VARCHAR(32)',
    'sexo': 'CHAR(1)',
    'idade': 'INT'
})

print(pessoa.create)
print(pessoa.drop)
print(pessoa.insert)
print(pessoa.select)
print(pessoa.update)
print(pessoa.delete)

CREATE TABLE 
  pessoa
(
  id INT NOT NULL AUTO_INCREMENT,
  nome VARCHAR(32),
  sexo CHAR(1),
  idade INT,
  PRIMARY KEY (id)
);
DROP TABLE pessoa;
INSERT INTO
  pessoa 
(
  nome,
  sexo,
  idade
)
VALUES
(
  @Nome,
  @Sexo,
  @Idade
);
SELECT
  id,
  nome,
  sexo,
  idade
FROM
  pessoa;
UPDATE
  pessoa
SET
 nome = @Nome,
 sexo = @Sexo,
 idade = @Idade
WHERE
  id = @Id;
DELETE FROM
  pessoa
WHERE
  id = @Id;
